In [1]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import math
try:
    import cPickle as pickle
except ImportError:
    import _pickle as pickle

In [2]:
with open('C:\\Users\\sara1\\Desktop\\Workingdata\\state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print (len(state_features))

['sofa_score', 'sirs', 'age', 'weight', 'gender', 'gcs', 'sofa_24hours', 'bilirubin_total', 'bicarbonate', 'hemoglobin', 'chloride', 'calcium', 'potassium', 'sodium', 'lactate', 'glucose', 'respiratory_rate_total', 'fio2', 'platelet', 'wbc', 'albumin', 'creatinine', 'heart_rate', 'temperature', 'spo2', 'vaso_rate', 'vaso_amount', 'urineoutput']
28


In [3]:
df = pd.read_csv('C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_data_final_cont.csv')

In [4]:
df_test = pd.read_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\rl_test_data_final_cont.csv")

In [5]:
df_val = pd.read_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_data_final_cont.csv")

In [6]:
n_input = len(state_features)
n_hidden_1 = 200
weights = {
    'encoder_h1': tf.Variable(tf.random.normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random.normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random.normal([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.random.normal([n_input])),
}

In [7]:
class AutoEncoder:
    def __init__(self):
        self.input_size = len(state_features)
        self.batch_size = tf.Variable(32, dtype=tf.int32)
        self.sparsity_dist = tf.fill([self.batch_size,n_hidden_1],0.05)
        self.X = tf.Variable("float", [None,self.input_size])
        self.layer_1_a = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['encoder_h1']),
                                   biases['encoder_b1']))
        self.layer_1_d = tf.nn.sigmoid(tf.add(tf.matmul(self.layer_1_a, weights['decoder_h1']),
                                   biases['decoder_b1']))
        self.target_y = self.X
        self.kl = tf.reduce_mean(tf.reduce_sum(tf.multiply(self.sparsity_dist,tf.log(self.sparsity_dist/self.layer_1_a)) + \
                                                tf.multiply((1-self.sparsity_dist),tf.log((1-self.sparsity_dist)/(1-self.layer_1_a))), axis =1))
        # regularisation term logic - loss at the start is roughly 60, of this 0.5 is reconstruction loss.  
        self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2)) + 0.0001*self.kl
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)

In [8]:
df.keys()

Index(['stay_id', 'sofa_score', 'sirs', 'age', 'weight', 'gender', 'gcs',
       'sofa_24hours', 'bilirubin_total', 'bicarbonate', 'hemoglobin',
       'chloride', 'calcium', 'potassium', 'sodium', 'lactate', 'glucose',
       'respiratory_rate_total', 'fio2', 'platelet', 'wbc', 'albumin',
       'creatinine', 'heart_rate', 'temperature', 'spo2', 'vaso_rate',
       'vaso_amount', 'urineoutput', 'died_in_hosp', 'vaso_input', 'reward'],
      dtype='object')

In [9]:
def eval_on_val():
    cur_state = df_val.ix[:,state_features]
    l = sess.run([autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size:len(df_val)})
    print ("validation loss is: ", l)

In [10]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [11]:
batch_size = 100
autoencoder = AutoEncoder()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for step in range(10000):
        sum_loss = 0
        a = df.sample(n=batch_size)
        cur_state = a.ix[:,state_features]
        _,l = sess.run([autoencoder.optimizer, autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size : batch_size})
        sum_loss += l
        if step % 1000==0 and step > 0:
            print ("step is ", step) 
            print ("total loss is ", sum_loss)
            sum_loss = 0
        if step % 10000 == 0 and step >0:
            b = df.sample(n=1)
            cur_state_trial = b.ix[:,state_features]
            target_y,reconstruction = sess.run([autoencoder.target_y,autoencoder.layer_1_d],feed_dict={autoencoder.X:cur_state_trial})
            print (reconstruction)
            print (target_y)
        if step % 10000 == 0 and step >0:
            eval_on_val()
    # run the model on the train, val, and test data to get a reduced dimensionality representation
    train_states = df.ix[:,state_features]
    train_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:train_states})
    train_autoencode = pd.DataFrame(train_repr[0])
    train_autoencode['vaso_input'] = df['vaso_input']
    train_autoencode['reward'] = df['reward']
    train_autoencode['stay_id'] = df['stay_id']
    
    val_states = df_val.ix[:,state_features]
    val_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:val_states})
    val_autoencode = pd.DataFrame(val_repr[0])
    val_autoencode['vaso_input'] = df_val['vaso_input']
    val_autoencode['reward'] = df_val['reward']
    val_autoencode['stay_id'] = df_val['stay_id']
    
    test_states = df_test.ix[:,state_features]
    test_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:test_states})
    test_autoencode = pd.DataFrame(test_repr[0])
    test_autoencode['vaso_input'] = df_test['vaso_input']
    test_autoencode['reward'] = df_test['reward']
    test_autoencode['stay_id'] = df_test['stay_id']

InvalidArgumentError: Value for attr 'T' of string is not in the list of allowed values: bfloat16, half, float, double, int32, int64, complex64, complex128
	; NodeDef: {{node MatMul}}; Op<name=MatMul; signature=a:T, b:T -> product:T; attr=transpose_a:bool,default=false; attr=transpose_b:bool,default=false; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_INT32, DT_INT64, DT_COMPLEX64, DT_COMPLEX128]> [Op:MatMul]

In [11]:
train_autoencode.to_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\rl_train_data_final_auto.csv", index=False)
val_autoencode.to_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_data_final_auto.csv", index=False)
test_autoencode.to_csv("C:\\Users\\sara1\\Desktop\\Workingdata\\rl_val_data_final_auto.csv", index=False)